In [42]:
from sympy import *
#from sympy.physics.mechanics import dynamicsymbols, kinetic_energy
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
import matplotlib.pyplot as plt
#import seaborn 

In [43]:
r_B, r_P, gamma_B, gamma_P = 6.2, 5, 0.2269, 0.82
phi, theta, psi = symbols ('phi theta psi')

In [44]:
c_phi = cos(phi); s_phi = sin(phi)
c_theta = cos(theta ); s_theta  = sin(theta )
c_psi = cos(psi); s_psi= sin(psi)

In [78]:
## Define the Geometry of the platform
# Psi_B (Polar coordinates)
psi_B = Matrix([[-gamma_B, 
    gamma_B,
    2*pi/3 - gamma_B, 
    2*pi/3 + gamma_B, 
    2*pi/3 + 2*pi/3 - gamma_B, 
    2*pi/3 + 2*pi/3 + gamma_B]])
psi_B.evalf()

# psi_P (Polar coordinates)
# Direction of the points where the rod is attached to the platform.
psi_P = Matrix([[
    pi/3 + 2*pi/3 + 2*pi/3 + gamma_P,
    pi/3 + -gamma_P, 
    pi/3 + gamma_P,
    pi/3 + 2*pi/3 - gamma_P, 
    pi/3 + 2*pi/3 + gamma_P, 
    pi/3 + 2*pi/3 + 2*pi/3 - gamma_P]])
psi_P.evalf()

# Coordinate of the points where servo arms 
# are attached to the corresponding servo axis.
B = r_B * Matrix( [ 
    [ cos(psi_B[0]), sin(psi_B[0]), 0],
    [ cos(psi_B[1]), sin(psi_B[1]), 0],
    [ cos(psi_B[2]), sin(psi_B[2]), 0],
    [ cos(psi_B[3]), sin(psi_B[3]), 0],
    [ cos(psi_B[4]), sin(psi_B[4]), 0],
    [ cos(psi_B[5]), sin(psi_B[5]), 0] ])
B = transpose(B).evalf()
#B = B.evalf()

# Coordinates of the points where the rods 
# are attached to the platform.
P = r_P * Matrix([ 
    [ cos(psi_P[0]), sin(psi_P[0]), 0],
    [ cos(psi_P[1]), sin(psi_P[1]), 0],
    [ cos(psi_P[2]), sin(psi_P[2]), 0],
    [ cos(psi_P[3]), sin(psi_P[3]), 0],
    [ cos(psi_P[4]), sin(psi_P[4]), 0],
    [ cos(psi_P[5]), sin(psi_P[5]), 0] ])
#P = transpose(P).evalf()
P = P.evalf()
B
#print('6x3 array for Base anchors \n', B)
#print('6x1 array for psi_B \n', psi_B.evalf())
#print('6x3 array for Platform anchors \n', P)
#print('6x1 array for psi_P \n', psi_P.evalf())

Matrix([
[ 6.04108436354526, 6.04108436354526, -1.81266189614963, -4.22842246739563, -4.22842246739563, -1.81266189614963],
[-1.39474001610656, 1.39474001610656,  5.92910253328843,  4.53436251718186, -4.53436251718186, -5.92910253328843],
[                0,                0,                 0,                 0,                 0,                 0]])

In [79]:
shape(B)

(3, 6)

In [73]:
# # Definition of the platform home position.
home_pos= Matrix([0, 0, 2*r_B])

# Allocate for variables
l = Matrix.zeros(3,6)
lll = Matrix.zeros(6)
# Rotation matrices used later
def rotX(phi):
    rotx = Matrix([[1, 0, 0],[0, cos(phi),-sin(phi)],[0, sin(phi),cos(phi)]])
    return rotx.evalf()

def rotY(phi):
    roty = Matrix([[cos(phi), 0,sin(phi)],[0, 1, 0],[-sin(phi), 0,cos(phi)]])
    return roty.evalf()

def rotZ(phi):
    rotz = Matrix([[cos(phi),-sin(phi),0],[sin(phi), cos(phi),0],[0, 0, 1]])
    return rotz.evalf()

In [76]:
shape(l)

(3, 6)

In [ ]:
import matplotlib.pyplot as plt
#import seaborn

def plot_2D_annotate(X, Y):
    X = list(X)
    Y = list(Y)
    ax.plot(X,Y, 'or')  # Plot Points
    # Annotate Points
    for i, (xy) in enumerate(zip(X, Y)):                                 
        ax.annotate('Point ' + str(i) + '\n(%.5s, %.5s)' % xy, xy=xy, textcoords='data')

    X.append(X[0])
    Y.append(Y[0])
    ax.plot(X,Y)    # Plot lines

#seaborn.set(style='ticks')
# plt.style.use('dark_background')

fig = plt.figure(figsize=(14, 6), dpi=80)

ax = fig.add_subplot(121)
ax.set_title('Base Anchors')
plot_2D_annotate(B[0,:], B[1,:])
circle_B = plt.Circle((0, 0), r_B, color='g', fill=False)
ax.add_patch(circle_B)

ax = fig.add_subplot(122)
ax.set_title('Platform Anchors')
plot_2D_annotate(P[0,:], P[1,:])
circle_r = plt.Circle((0, 0), r_P, color='g', fill=False)
ax.add_patch(circle_r)

plt.show()

In [41]:
# Given input trans, rotation
#trans = Transpose(Matrix([0,0,0])) # X, Y, Z
trans = Matrix([0,0,0]) # X, Y, Z
#rotation = Transpose(Matrix([0,0,pi/6])) # X, Y, Z
rotation = Matrix([0,0,pi/6]) # X, Y, Z

R = rotX(rotation[0])*rotY(rotation[1])*rotZ(rotation[2])

# Get rotation matrix of platform. RotZ* RotY * RotX -> matmul
# R = np.matmul( np.matmul(rotZ(rotation[2]), rotY(rotation[1])), rotX(rotation[0]) )
#R = np.matmul( np.matmul(rotX(rotation[0]), rotY(rotation[1])), rotZ(rotation[2]) )

# Get leg length for each leg
# leg = np.repeat(trans[:, np.newaxis], 6, axis=1) + np.repeat(home_pos[:, np.newaxis], 6, axis=1) + np.matmul(np.transpose(R), P) - B 

##l = np.repeat(trans[:, np.newaxis], 6, axis=1) + np.repeat(home_pos[:, np.newaxis], 6, axis=1) + np.matmul(R, P) - B 
#l = trans + home_pos + (P.cross(R)) - B
#l = trans + home_pos + (P.cross(R)) - B
##lll = np.linalg.norm(l, axis=0)
P.evalf()
# Position of leg in global frame
##L = l + B

##print('Leg lengths to command in order to achieve desired position of plate: \n', lll)

Matrix([
[ 4.87150733029175, -1.12623990824057, 0],
[ 4.87150733029175,  1.12623990824057, 0],
[-1.46040129385368,  4.78196905687505, 0],
[-3.41110603643807,  3.65572914863448, 0],
[-3.41110603643807, -3.65572914863448, 0],
[-1.46040129385368, -4.78196905687505, 0]])

In [ ]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot3D_line(ax, vec_arr_origin, vec_arr_dest, color_):
    for i in range(6):
        ax.plot([vec_arr_origin[0, i] , vec_arr_dest[0, i]],
        [vec_arr_origin[1, i], vec_arr_dest[1, i]],
        [vec_arr_origin[2, i],vec_arr_dest[2, i]],
        color=color_)

#interactive plotting in separate window
# Comment out this and restart kernel if you're having issues with interactive plot (qt)
# %matplotlib qt

ax = plt.axes(projection='3d') # Data for a three-dimensional line
ax.set_xlim3d(-10, 10)
ax.set_ylim3d(-10, 10)
ax.set_zlim3d(0, 20)

# ax.add_collection3d(Poly3DCollection([list(np.transpose(s.B))]), zs='z')
ax.add_collection3d(Poly3DCollection([list(Transpose(B))], facecolors='green', alpha=0.25))

# ax.add_collection3d(base_plot, zs='z')
ax.add_collection3d(Poly3DCollection([list(Transpose(L))], facecolors='blue', alpha=0.25))

plot3D_line(ax, B, L, 'orange')
plt.show()